# SQL Package

Provides simple functionality to interact with a PostgreSQL server using Python classes.

**Overview of functionality:**
* Database(self, user, password, host, dbname, port)
    * properties
        * user
        * password
        * host
        * dbname
        * port
    * methods
        * create(name) x
        * connect()
        * drop(name)
* Table(self, dbname, table, schema)
    * accepts db properties
    * properties
        * connect() --> inherited
        * fetch_data(sql, con, parse_dates)
        * get_names()
        * format_names(char_dict)
        * update_names(names_dict)
        * add_columns(columns_list, type=None)
        * compare_column_order(dataframe)
        * match_columns(dataframe)
        * save_csv(data, local_path, match_column_order=True)
        * update_values(local_path, container_path)
        * update_types(types_dict)
        * close()

## Setup

In [1]:
import os
import sys
from pathlib import Path
#sys.path[0] = str(Path(__file__).resolve().parents[2]) # Set path for custom modules
import warnings
from io import StringIO

# Set path for modules
sys.path[0] = '../'

from dotenv import load_dotenv, find_dotenv
import numpy as np
import pandas as pd

# SQL libraries
import psycopg2
from src.pipeline.dictionaries import types_dict, replace_map
from src.pipeline.transform_data import create_full_address, split_lat_long
from src.toolkits.geospatial import geocode_from_address
from src.toolkits.postgresql import Database, Table
from src.toolkits.eda import explore_value_counts

# Set notebook display options
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

# Get project root directory
#root_dir = os.path.dirname(os.getcwd())

# if modulename not in sys.modules: print...
load_dotenv(find_dotenv());

In [2]:
types_dict_abbrev = {'assessor_book': 'SMALLINT',
             'assessor_page': 'SMALLINT',
             'assessor_parcel': 'CHAR(3)',
             'tract': 'VARCHAR(200)',
             'block': 'VARCHAR(50)',
             'lot': 'VARCHAR(50)',
             'reference_no_old_permit_no': 'VARCHAR(50)',
             'pcis_permit_no': 'VARCHAR(50)',
             'status': 'VARCHAR(50)',
             'status_date': 'DATE',
             'permit_type': 'VARCHAR(50)',
             'permit_sub_type': 'VARCHAR(50)',
             'permit_category': 'VARCHAR(50)',
             'project_number': 'SMALLINT',
             'event_code': 'VARCHAR(50)',
             'initiating_office': 'VARCHAR(50)',
             'issue_date': 'DATE',
             'address_start': 'INTEGER',
             'address_fraction_start': 'CHAR(3)',
             'address_end': 'INTEGER',
             'address_fraction_end': 'CHAR(3)',
             'street_direction': 'CHAR(1)',
             'street_name': 'VARCHAR(50)',
             'street_suffix': 'VARCHAR(10)',
             'suffix_direction': 'VARCHAR(10)',
             'unit_range_start': 'VARCHAR(50)',
             'unit_range_end': 'VARCHAR(50)',
             'zip_code': 'INTEGER',
             'work_description': 'TEXT',
             'valuation': 'NUMERIC',
             'floor_area_la_zoning_code_definition': 'VARCHAR(10)',
             'no_of_residential_dwelling_units': 'SMALLINT',
             'no_of_accessory_dwelling_units': 'SMALLINT',
             'no_of_stories': 'SMALLINT',
             'contractors_business_name': 'VARCHAR(100)',
             'contractor_address': 'VARCHAR(100)',
             'contractor_city': 'VARCHAR(50)',
             'contractor_state': 'CHAR(2)',
             'license_type': 'VARCHAR(10)',
             'license_no': 'INTEGER',
             'principal_first_name': 'VARCHAR(50)',
             'principal_middle_name': 'VARCHAR(50)',
             'principal_last_name': 'VARCHAR(50)',
             'license_expiration_date': 'DATE',
             'applicant_first_name': 'VARCHAR(50)',
             'applicant_last_name': 'VARCHAR(50)',
             'applicant_business_name': 'VARCHAR(100)',
             'applicant_address_1': 'VARCHAR(50)',
             'applicant_address_2': 'VARCHAR(50)',
             'applicant_address_3': 'VARCHAR(50)',
             'zone': 'VARCHAR(50)',
             'occupancy': 'VARCHAR(50)',
             'floor_area_la_building_code_definition': 'VARCHAR(10)',
             'census_tract': 'VARCHAR(10)',
             'council_district': 'SMALLINT',
             'latitude_longitude': 'VARCHAR(50)',
             'applicant_relationship': 'VARCHAR(50)',
             'existing_code': 'SMALLINT',
             'proposed_code': 'SMALLINT'}

In [3]:
permits = Database()

In [4]:
#params = {"table_name":"permits_raw", "types_dict":types_dict_abbrev, "id_col":"pcis_permit_no"}
#permits.drop_table('permits_raw').drop_table('tmp_permits_raw').create_table(**params)
#!cd ../ && bash scripts/load_db.sh

In [5]:
permits.list_tables()

['permits_raw']

### Run Pipeline

1. Standardize table names.<br>
2. Fetch raw data and transform:
    - Concatenate address columns
    - Geocode missing coordinates using street address
    - Extract latitude and longitude from coordinates into their own columns
3. Update the table with the transformed data:
    - Add new columns to PostgreSQL table
    - Update values

In [6]:
permits_raw = Table(name="permits_raw")

#### Standardize table names

In [7]:
permits_raw.format_table_names(replace_map=replace_map, update=True)

Query successful on database "permits".


In [8]:
permits_raw.update_types(types_dict)

Query successful on database "permits".


#### Fetch and Transform

In [9]:
data = permits_raw.fetch_data()

In [10]:
data = create_full_address(data)

In [11]:
geocode_from_address(data);

Cost for geocoding 19 addresses is $0.10.
Geocoding...
19 locations were assigned coordinates.


In [12]:
data = split_lat_long(data)

#### Update Table

In [13]:
permits_raw.update_values(data=data, id_col="pcis_permit_no", types_dict=types_dict)

Query successful on database "permits".
Query successful on database "permits".
Query successful on database "permits".
Rearranged dataframe columns to match table "permits_raw".
Rearranged dataframe columns to match table "permits_raw".
Copy successful on table "permits_raw".
Query successful on database "permits".
